In [1]:
import pandas as pd
import numpy as np

In [47]:
train = pd.read_csv("../MachineHackODIMatchWinner/Train.csv")
test = pd.read_csv("../MachineHackODIMatchWinner/Test.csv")
sf = pd.read_excel("../MachineHackODIMatchWinner/Sample_submission.xlsx")

In [48]:
def normalabelencoder(train,test,cols):
    from sklearn.preprocessing import LabelEncoder

    label_enc = LabelEncoder()
    
    print(cols)
    for col in cols:
        label_enc.fit(train[col])
        train[col] = label_enc.transform(train[col])
        test[col] = label_enc.transform(test[col])
    
    return train,test


In [49]:
def onhotencoder(train,test,cols):
    
    from sklearn.preprocessing import OneHotEncoder
    #object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    object_cols=cols
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

In [50]:
trainlblohe ,testlblohe = normalabelencoder(train,test, cols = [ 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch']) 

['Team1_Venue', 'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch']


In [51]:
y = trainlblohe.MatchWinner

X = trainlblohe.drop(['MatchWinner'], axis=1)


In [52]:
 from sklearn.ensemble import RandomForestClassifier

In [58]:
from sklearn.calibration import CalibratedClassifierCV

# Create the example dataset and split it.
np.random.seed(42)



In [71]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_sample(X, y)


Using TensorFlow backend.


In [72]:
clf2= RandomForestClassifier(random_state=2)


clf_sigmoid = CalibratedClassifierCV(clf2, cv=5, method='sigmoid')
clf_sigmoid.fit(X_ros, y_ros)
preds = clf_sigmoid.predict_proba(testlblohe)
    

In [73]:
preds.shape

(1075, 16)

In [74]:
# clf_sigmoid.feature_importances_

In [75]:
sf = pd.read_excel("../MachineHackODIMatchWinner/Sample_submission.xlsx")
prediction = pd.DataFrame(data =preds, columns=sf.columns)
prediction.to_excel('../MachineHackODIMatchWinner/cajokesigmoidros.xlsx')


In [76]:
prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.002348,0.015994,0.888352,0.000829,0.011880,0.013816,0.002459,0.001203,0.000793,0.014971,0.013200,0.000759,0.007731,0.009440,0.010510,0.005714
1,0.003062,0.743703,0.004112,0.001128,0.047662,0.017785,0.003224,0.001649,0.001075,0.013082,0.018207,0.001002,0.012184,0.013636,0.110140,0.008348
2,0.002287,0.014721,0.002146,0.000836,0.010553,0.013579,0.002420,0.001187,0.000780,0.013296,0.902405,0.000745,0.006844,0.010766,0.011786,0.005648
3,0.003155,0.022251,0.003135,0.001173,0.014484,0.019463,0.003492,0.001748,0.001113,0.315781,0.525802,0.001098,0.009706,0.019463,0.050118,0.008019
4,0.004591,0.030142,0.003695,0.001375,0.015603,0.676494,0.003971,0.001915,0.001311,0.015671,0.093106,0.001266,0.010688,0.022869,0.039899,0.077405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0.002293,0.014896,0.002138,0.000827,0.010532,0.884245,0.002446,0.001202,0.000789,0.008991,0.013924,0.000753,0.006621,0.012230,0.010814,0.027298
1071,0.002696,0.632589,0.002334,0.000931,0.011060,0.014190,0.002734,0.001363,0.000897,0.012046,0.015686,0.000852,0.271722,0.012369,0.012310,0.006221
1072,0.002539,0.016321,0.002259,0.000882,0.010500,0.862866,0.002593,0.001277,0.000845,0.019710,0.037409,0.000806,0.013090,0.011388,0.011262,0.006253
1073,0.005165,0.031933,0.004541,0.001772,0.020592,0.032246,0.005305,0.002546,0.001700,0.032879,0.386263,0.001634,0.165849,0.266371,0.029094,0.012111
